In [1]:
import torchaudio
from demucs import pretrained
from demucs.apply import apply_model
import soundfile as sf
import torch
import os
import matplotlib.pyplot as plt


def plot_waveforms(original, denoised, sr, output_plot_path):
    plt.figure(figsize=(14, 6))

    plt.subplot(2, 1, 1)
    plt.title("Original Waveform")
    plt.plot(original.t().numpy())
    plt.xlabel("Time")
    plt.ylabel("Amplitude")

    plt.subplot(2, 1, 2)
    plt.title("Denoised Waveform")
    plt.plot(denoised.t().numpy())
    plt.xlabel("Time")
    plt.ylabel("Amplitude")

    plt.tight_layout()
    plt.savefig(output_plot_path)
    plt.close()


def denoise_with_demucs(input_wav_path: str, output_wav_path: str, output_plot_path: str):
    # Load audio (preserve sample rate and stereo)
    wav, sr = torchaudio.load(input_wav_path)

    # Make sure tensor is 2D (channels, time)
    if wav.dim() == 1:
        wav = wav.unsqueeze(0)

    # Load Demucs pre-trained model (v4)
    model = pretrained.get_model(name="htdemucs")

    # Run model
    with torch.no_grad():
        sources = apply_model(model, wav, samplerate=sr, device="cuda" if torch.cuda.is_available() else "cpu")

    # Extract vocals (speech signal)
    vocals = sources["vocals"]

    # Save denoised audio
    vocals_np = vocals.cpu().squeeze().numpy()
    sf.write(output_wav_path, vocals_np.T, sr)

    # Plot and save waveform comparison
    plot_waveforms(wav, vocals, sr, output_plot_path)


if __name__ == "__main__":
    input_path = "input.wav"                   # Replace with your file
    output_path = "denoised_output.wav"       # Output WAV path
    output_plot = "waveform_comparison.png"    # Output plot path

    denoise_with_demucs(input_path, output_path, output_plot)
    print(f"Denoised audio saved to: {output_path}")
    print(f"Waveform comparison plot saved to: {output_plot}")


ModuleNotFoundError: No module named 'demucs'